In [ ]:
Where are we currently in the repository?

In [1]:
!pwd

/home/jgutman/mvesc/Descriptives/cohort_analysis


In [2]:
!git branch

  clean_grades_and_attendance_tables
* cohort_analysis
  consolidating_tables
  import_text_files_issue1
  master
  track_students_descriptives


In [3]:
!ls /home/jgutman/mvesc/ETL

all_absences_generate_mm_day_wkd.sql  etl_test_write_ZZ.ipynb
building_cohort_analysis.sql	      file_to_table_name.json
build_student_tracking.py	      fill_missing_years.py
clean_absences.sql		      module_dev_utility_functions.ipynb
clean_oaaogt_0616.sql		      mvesc_utility_functions.py
csv2postgres_mvesc.py		      __pycache__
ETL_test_and_dev_XC.ipynb	      snapshot_column_names.json


In [4]:
import os, sys, imp
parentdir = os.path.abspath('/home/jgutman/mvesc/ETL')
sys.path.insert(0,parentdir) 

In [5]:
import build_student_tracking as track
from mvesc_utility_functions import postgres_pgconnection_generator

In [11]:
imp.reload(track)

<module 'build_student_tracking' from '/home/jgutman/mvesc/ETL/build_student_tracking.py'>

In [12]:
with postgres_pgconnection_generator() as connection:
        with connection.cursor() as cursor:
            # print(track.sql_gen_tracking_students(2006, 2015))
            # print(track.cohort_survival_analysis(2006, 2015, '04'))
            print(track.build_wide_format(cursor, '09', 2010))
            get_graduates_query = """
        select student_lookup, graduation_date from 
            (select student_lookup, max(school_year) as school_year from clean.all_snapshots 
                where student_lookup in 
                    (select distinct student_lookup from clean.all_snapshots 
                        where graduation_date is not null) 
                and graduation_date is not null
            group by student_lookup) as latest_grade_with_graduation
        left join 
            (select student_lookup, school_year, graduation_date from clean.all_snapshots
                where graduation_date is not null) as graduation_dates_valid
        using (student_lookup, school_year)
            """
            cursor.execute(get_graduates_query)
            graduates = cursor.fetchall()
            print(len(graduates))
        connection.commit()

['grade', 'school_year', 'count']
[(9, 2010, 1397), (10, 2011, 1034), (11, 2012, 964), (12, 2013, 923)]
9222


In [18]:
import pandas as pd
pd.__version__

'0.18.1'

In [19]:
import igraph
from igraph import *
import cairo
from mvesc_utility_functions import postgres_pgconnection_generator

In [29]:
with postgres_pgconnection_generator() as connection:
        with connection.cursor() as cursor:
            cohort_tree = build_empty_tree()
            cohort_tree = get_bucket_counts(
                cursor, cohort_tree, grade_begin = '09', year_begin = 2010)
        connection.commit()

IGRAPH U--- 18 17 --
+ attr: count (v), description (v), outcomes (v), students (v)
+ edges:
0--1 0--2 1--3 1--4 2--5 2--6 2--7 3--8 3--9 4--10 4--11 4--12 11--13 11--14
14--15 14--16 14--17


In [21]:
def build_empty_tree():
    """
    Builds an empty tree obj for outcome classification. Vertices are named and
    counts for each vertex are initialized at zero. Pass tree along with cursor
    to get_bucket_counts method to fill in student counts at each vertex.

    Tree contains 18 vertices and 17 edges
    4 vertex attributes: description, count, outcome, and students
    description: the fine-grained description of the bucket
    outcome: the rough category of the bucket
        (non-terminal, on-time, late, dropout, exclude, uncertain)
    count: the number of students in each bucket
    students: a list of student lookups in each bucket
    """

    # Initialize tree structure
    num_vertices = 18
    tree = Graph()
    tree.add_vertices(num_vertices)
    tree.add_edges([(0,1), (0,2), (1,3), (1,4), (2,5), (2,6), (2,7),
        (3,8), (3,9), (4,10), (4,11), (4,12), (11,13), (11,14),
        (14,15), (14,16), (14,17)])
    tree.vs["description"] = ["cohort total", "no graduation date",
        "graduation date", "no withdrawal reason", "withdrawal reason",
        "4 year graduation", "5 year graduation", "more than 5 years",
        "no withdrawal date", "district withdrawal date", "misc withdrawal",
        "transferred", "dropout", "no withdraw to IRN", "withdrawn to IRN",
        "dropout recovery program", "JVSD/career tech", "other Ohio IRN"]
    assert(len(tree.vs["description"]) == num_vertices)
    tree.vs["count"] = [0] * num_vertices
    tree.vs["students"] = [None] * num_vertices

    # Map fine-grained bucket descriptions to rough outcome categories
    outcome_buckets = {}
    outcome_buckets["non-terminal"] = ["cohort total", "no graduation date",
        "graduation date", "no withdrawal reason", "withdrawal reason",
        "transferred", "withdrawn to IRN"]
    outcome_buckets["exclude"] = ["misc withdrawal"]
    outcome_buckets["dropout"] = ["dropout", "dropout recovery program"]
    outcome_buckets["uncertain"] = ["no withdrawal date",
        "district withdrawal date", "no withdraw to IRN", "JVSD/career tech",
        "other Ohio IRN"]
    outcome_buckets["late"] = ["5 year graduation", "more than 5 years"]
    outcome_buckets["on-time"] = ["4 year graduation"]
    # Reverse keys and values of outcome_buckets dict
    outcome_buckets_flipped = dict((outcome,
            [k for k,v in outcome_buckets.items() if outcome in v][0])
            for outcome in tree.vs["description"])
    # Set tree vertex attributes for broad categories defined above
    tree.vs["outcomes"] = [outcome_buckets_flipped[bucket]
            for bucket in tree.vs["description"]]

    ### putting some basic useful commands here temporarily for reference
    print(tree) # 18 vertices, 17 edges, 4 vertex attributes, 0 edge attributes
    return tree

In [28]:
def get_bucket_counts(cursor, tree, grade_begin, year_begin,
        schema = 'clean', table = 'wrk_tracking_students'):
    # Think about how to do these queries nested recursively
    
    # First query: total students in cohort
    cohort_total_query = """select distinct student_lookup from 
        {schema}.{table} where "{year_begin}" = '{grade_begin}'
    """.format(schema=schema, table=table, year_begin=year_begin,
        grade_begin=grade_begin)
    update_tree_with_query(cursor, tree, cohort_total_query, "cohort total")
    print(tree.vs[cohort_index].attributes())
    
    
    return tree

In [ ]:
def update_tree_with_query(cursor, tree, query, desc_label):
    cursor.execute(query)
    student_list = cursor.fetchall()
    vertex_list = tree.vs.select(description=desc_label)
    vertex_index = [v.index for v in vertex_list][0]
    tree.vs[vertex_index]["students"] = student_list
    tree.vs[vertex_index]["count"] = len(student_list)
    assert([v["count"] for v in vertex_list][0] == len(student_list))
    assert([v["students"] for v in vertex_list][0] == student_list)